# Clone data and dependencies

In [ ]:
!pip install underthesea
!pip install nltk
!pip install transformers
!pip install cython
!pip install youtokentome
!pip install datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 4.9 MB/s 
     |████████████████████████████████| 581 kB 56.3 MB/s 
     |████████████████████████████████| 965 kB 65.1 MB/s 
     |████████████████████████████████| 235 kB 74.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.7 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 69.8 MB/s 
     |████████████████████████████████| 6.6 MB 50.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/sim

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd /gdrive/My\ Drive/Group\ 3 - NLP\ Project/source_code

/gdrive/My Drive/NLP_Project


In [ ]:
from src.vocabulary import Vocabulary
from src.dataset import StanfordEnViDataset,get_loader
from src.model import EnViTransformer
from src.utils import save_checkpoint, load_checkpoint, training_step, validate, beam_search
from src.Translator import TransformerTranslator

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import yaml
from yaml.loader import SafeLoader
import torch
import datetime
with open('./utils/config/word_segmentation_config.yml') as f:
    word_segment_config = yaml.load(f, Loader=SafeLoader)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

segmentTranslator = TransformerTranslator(word_segment_config, device, input_type = 'word_segmentation')
print(len(segmentTranslator.vi_vocab), len(segmentTranslator.en_vocab))


16250 22407


In [ ]:
segmentTranslator.train()

**************************************************************************    Epoch number 1    **************************************************************************


  0%|          | 0/2084 [00:00<?, ?it/s]

Current step: 1, epoch: 1    Training loss:  9.869293212890625    Evaluation loss:  9.867773475646972    Time elapsed:  0:00:03.314479    (Best weights saved!) 
Current step: 250, epoch: 1    Training loss:  8.541838222503662    Evaluation loss:  7.189964294433594    Time elapsed:  0:01:30.896494    (Best weights saved!) 
Current step: 500, epoch: 1    Training loss:  7.398116600990296    Evaluation loss:  5.7500807762146    Time elapsed:  0:02:58.400769    (Best weights saved!) 


In [ ]:
segmentTranslator.inference(['I am a student at Cambridge University', 'My family was not poor , and myself , I had never experienced hunger since 2002 .'
, 'Kim Jong Un is the president of Korea'])

In [ ]:
import underthesea
en_test_sents = []
with open("./data/processed/test/tst2013.en") as f:
    for sent in f:
        sent  = sent.strip()
        en_test_sents.append(sent)

vi_test_sents = []
with open("./data/processed/test/tst2013.vi") as f:
    for sent in f:
        sent  = sent.strip()
        if sent[-2:] == " .":
            sent = sent[:-2] + "."
        if sent[-2:] == " ?":
            sent = sent[:-2] + "?"
        if sent[-2:] == " !":
            sent = sent[:-2] + "!"

        vi_test_sents.append(sent)


In [ ]:
translations = bpeTranslator.inference(en_test_sents)

In [ ]:
! pip install sacrebleu

In [ ]:
from sacrebleu.metrics import BLEU
def calc_bleu(targets, translations, case_sensitive = False):
  if case_sensitive:
    print("Calculating case sensitive BLEU")
    targets = [sentence.strip() for sentence in targets]
    sys = [translation.strip() for translation in translations] 
  else:
    print("Calculating case insensitive BLEU")
    targets = [sentence.strip().lower() for sentence in targets]
    sys = [translation.strip().lower() for translation in translations] 

  refs = [targets]
  bleu = BLEU()
  a = bleu.corpus_score(sys, refs)
  print(a)

In [ ]:
calc_bleu(vi_test_sents, translations, case_sensitive = False)
calc_bleu(vi_test_sents, translations, case_sensitive = True)